# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
G1 = nx.read_gpickle("assets/A4_P1_G1")
G2 = nx.read_gpickle("assets/A4_P1_G2")
G3 = nx.read_gpickle("assets/A4_P1_G3")
G4 = nx.read_gpickle("assets/A4_P1_G4")
G5 = nx.read_gpickle("assets/A4_P1_G5")
P1_Graphs = [G1, G2, G3, G4, G5]

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs using any methodology and determine which of the three algorithms generated each graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [14]:
def graph_identification():
    # YOUR CODE HERE
    classifications = []
    
    for G in P1_Graphs:
        degree_sequence = [d for _, d in G.degree()]
        avg_clustering = nx.average_clustering(G)
        avg_shortest_path = nx.average_shortest_path_length(G)
        max_degree = max(degree_sequence)

        # Identify the graph type based on the analyzed thresholds
        if max_degree > 30 and avg_clustering < 0.1:  # Power-law distribution (Preferential Attachment)
            classifications.append("PA")
        elif avg_clustering > 0.45 and avg_shortest_path > 35:  # High clustering and long path (SW_L)
            classifications.append("SW_L")
        else:  # Shorter path length, moderate clustering (SW_H)
            classifications.append("SW_H")

    return classifications
        
    raise NotImplementedError()
    
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

In [15]:
# import matplotlib.pyplot as plt

# def analyze_graphs(graphs):
#     for i, G in enumerate(graphs):
#         # Compute metrics
#         degree_sequence = [d for _, d in G.degree()]
#         avg_clustering = nx.average_clustering(G)
#         avg_shortest_path = nx.average_shortest_path_length(G)

#         # Print metrics
#         print(f"Graph {i+1}:")
#         print(f"  - Max Degree: {max(degree_sequence)}")
#         print(f"  - Avg Clustering Coefficient: {avg_clustering:.4f}")
#         print(f"  - Avg Shortest Path Length: {avg_shortest_path:.4f}")

#         # Plot degree distribution
#         plt.figure()
#         plt.hist(degree_sequence, bins=range(min(degree_sequence), max(degree_sequence) + 1), alpha=0.75, color='blue')
#         plt.title(f"Degree Distribution of Graph {i+1}")
#         plt.xlabel("Degree")
#         plt.ylabel("Frequency")
#         plt.show()

# # Run analysis on the five graphs
# analyze_graphs(P1_Graphs)

In [16]:
ans_one = graph_identification()
assert type(ans_one) == list, "You must return a list"


---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [17]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features of your choice using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a Pandas series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [18]:
list(G.nodes(data=True))[:5] # print the first 5 nodes

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan})]

In [29]:
def salary_predictions():
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score

    # YOUR CODE HERE
    # Assuming G is already defined (e.g., via pickle.load)
    nodes = list(G.nodes())
    
    # Compute network features.
    degree_dict = dict(G.degree())
    clustering_dict = nx.clustering(G)
    avg_neighbor_dict = nx.average_neighbor_degree(G)
    pagerank_dict = nx.pagerank(G)
    
    # Build a DataFrame (rows correspond to nodes)
    df = pd.DataFrame(index=nodes)
    df['degree'] = pd.Series(degree_dict)
    df['clustering'] = pd.Series(clustering_dict)
    df['avg_neighbor_degree'] = pd.Series(avg_neighbor_dict)
    df['pagerank'] = pd.Series(pagerank_dict)
    
    # Add the Department attribute (provided as an integer) and one-hot encode it.
    dept_dict = {n: G.nodes[n].get('Department') for n in nodes}
    df['Department'] = pd.Series(dept_dict)
    df = pd.get_dummies(df, columns=['Department'], prefix='Dept')
    
    # Add the target variable: ManagementSalary.
    salary_dict = {n: G.nodes[n].get('ManagementSalary') for n in nodes}
    df['ManagementSalary'] = pd.Series(salary_dict)
    
    # Split into training and test sets:
    #  - Training: nodes where ManagementSalary is known.
    #  - Test: nodes where ManagementSalary is missing.
    train_df = df[df['ManagementSalary'].notnull()]
    test_df = df[df['ManagementSalary'].isnull()]
    
    feature_cols = [col for col in df.columns if col != 'ManagementSalary']
    X_train = train_df[feature_cols]
    y_train = train_df['ManagementSalary'].astype(int)
    X_test = test_df[feature_cols]
    
    # Identify numeric features vs. one-hot encoded features.
    numeric_cols = ['degree', 'clustering', 'avg_neighbor_degree', 'pagerank']
    onehot_cols = [col for col in feature_cols if col not in numeric_cols]
    
    # Scale the numeric features.
    scaler = StandardScaler()
    X_train_numeric = pd.DataFrame(scaler.fit_transform(X_train[numeric_cols]),
                                   index=X_train.index, columns=numeric_cols)
    X_test_numeric = pd.DataFrame(scaler.transform(X_test[numeric_cols]),
                                  index=X_test.index, columns=numeric_cols)
    
    # Recombine the scaled numeric features with the one-hot features.
    X_train_processed = pd.concat([X_train_numeric, X_train[onehot_cols]], axis=1)
    X_test_processed = pd.concat([X_test_numeric, X_test[onehot_cols]], axis=1)
    
    
    # Evaluate classifier performance via train/validation split.
    X_train_cv, X_val, y_train_cv, y_val = train_test_split(
        X_train_processed, y_train, test_size=0.2, random_state=42
    )
    
    clf_cv = RandomForestClassifier(random_state=42, n_estimators=100)
    clf_cv.fit(X_train_cv, y_train_cv)
    y_val_proba = clf_cv.predict_proba(X_val)[:, 1]
    auc_score = roc_auc_score(y_val, y_val_proba)
    print("Validation ROC AUC Score:", auc_score)
    
    # Train on the full training set.
    clf = RandomForestClassifier(random_state=42, n_estimators=100)
    clf.fit(X_train_processed, y_train)
    
    # Predict probability (for class 1: receiving management salary) for missing nodes.
    y_proba = clf.predict_proba(X_test_processed)[:, 1]
    
    # Return predictions as a Pandas Series with node id as index.
    pred_series = pd.Series(y_proba, index=X_test_processed.index)
    return pred_series
    
    raise NotImplementedError()
    
salary_predictions()

Validation ROC AUC Score: 0.9456391875746715


1      0.01
65     0.89
18     0.11
215    0.38
283    0.89
       ... 
691    0.00
788    0.02
944    0.00
798    0.02
808    0.00
Length: 252, dtype: float64

In [28]:
ans_salary_preds = salary_predictions()
assert type(ans_salary_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_salary_preds) == 252, "The series must be of length 252"


AUC Score: 1.0000


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [21]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to:      
1. Create a matrix of features of your choice for the edges found in `future_connections` using Networkx     
2. Train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data     
3. Predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [31]:
def new_connections_predictions():
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score

    # YOUR CODE HERE
    # Create a list of candidate edges (each candidate is a tuple (u, v))
    candidates = list(future_connections.index)
    
    # Pre-compute Jaccard coefficients
    # This returns a generator; we convert it to a dictionary.
    jaccard_dict = { (u, v): jaccard 
                     for u, v, jaccard in nx.jaccard_coefficient(G, candidates) }
    
    # Pre-compute Preferential Attachment
    pref_dict = { (u, v): pa 
                  for u, v, pa in nx.preferential_attachment(G, candidates) }
    
    # Initialize dictionaries to hold our features.
    feat_dict = {}       # For common neighbors, Jaccard coefficient, preferential attachment.
    same_dept_dict = {}  # Flag: 1 if both nodes are in the same department, else 0.
    
    # Loop over each candidate edge and compute features.
    for candidate in candidates:
        u, v = candidate
        # Compute number of common neighbors.
        cn = len(list(nx.common_neighbors(G, u, v)))
        # Retrieve the Jaccard coefficient from the pre-computed dictionary.
        jaccard = jaccard_dict.get(candidate, 0)
        # Retrieve the preferential attachment score from the pre-computed dictionary.
        pa = pref_dict.get(candidate, 0)
        
        feat_dict[candidate] = (cn, jaccard, pa)
        # Compute same department flag.
        dept_u = G.nodes[u]['Department']
        dept_v = G.nodes[v]['Department']
        same_dept_dict[candidate] = 1 if dept_u == dept_v else 0

    # Build a DataFrame from the features.
    df_features = pd.DataFrame.from_dict(feat_dict, orient='index', 
                                         columns=['common_neighbors', 'jaccard', 'pa'])
    df_features['same_department'] = pd.Series(same_dept_dict)
    
    # Compute the Adamic–Adar index for each candidate edge.
    adamic_scores = {}
    for u, v, score in nx.adamic_adar_index(G, candidates):
        adamic_scores[(u, v)] = score
    df_features['adamic'] = pd.Series(adamic_scores)
    
    # Now, df_features is our feature matrix for every candidate edge.
    # Determine the training set as those candidate edges with known Future Connection values.
    known_mask = future_connections['Future Connection'].notnull()
    X_train = df_features.loc[future_connections.index[known_mask]]
    # Convert labels (which are 0.0 or 1.0) to integers.
    y_train = future_connections.loc[future_connections.index[known_mask], 'Future Connection'].astype(int)
    
    # Our goal is to predict probabilities for all candidate edges.
    X_all = df_features.copy()
    
    # Scale the features (all features are numeric).
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_all_scaled = scaler.transform(X_all)
    
    
    # Evaluate classifier performance via train/validation split on the known edges.
    X_train_cv, X_val, y_train_cv, y_val = train_test_split(
        X_train_scaled, y_train, test_size=0.2, random_state=42
    )
    clf_cv = RandomForestClassifier(random_state=42, n_estimators=100)
    clf_cv.fit(X_train_cv, y_train_cv)
    y_val_pred = clf_cv.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, y_val_pred)
    print("Validation ROC AUC Score:", roc_auc)
    
    
    # Train a Random Forest classifier on all known edges.
    clf = RandomForestClassifier(random_state=42, n_estimators=100)
    clf.fit(X_train_scaled, y_train)
    
    # Predict the probability that each candidate edge is a future connection.
    proba = clf.predict_proba(X_all_scaled)[:, 1]
    
    # Create a series from the predictions.
    pred_series = pd.Series(proba, index=X_all.index)
    
    # Now, filter to only edges with missing Future Connection values.
    missing_edges = future_connections[future_connections['Future Connection'].isnull()].index
    pred_series = pred_series.loc[missing_edges]
    
    return pred_series
    raise NotImplementedError()
    
new_connections_predictions()

Validation ROC AUC Score: 0.8921505438193384


(107, 348)    0.000000
(542, 751)    0.012234
(20, 426)     0.240000
(50, 989)     0.014604
(942, 986)    0.011112
                ...   
(165, 923)    0.016459
(673, 755)    0.009350
(939, 940)    0.011112
(555, 905)    0.018244
(75, 101)     0.000000
Length: 122112, dtype: float64

In [26]:
ans_prob_preds = new_connections_predictions()
assert type(ans_prob_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_prob_preds) == 122112, "The series must be of length 122112"


AUC Score: 0.8937


AssertionError: The series must be of length 122112